# WISCS
> Words and images in shared conceptual space

**NOTE**: `power` branch contains modified code. A major refactor is pending and will be merged with `main` ASAP.

## Imports
A specalized module, `wiscs`, has been created to generate data that aligns with the various hypothesis proposed in this study.

In [1]:
from wiscs import simulate, make_tasks, print_deltas

In [5]:
design = dict(questions=True, items=True)
word_params = {'encoding': 200, 'decision': 50}
image_params = {'encoding': 150, 'decision': 70}
noise_params = {'word': 10, 'image': 10, 'participant': 30, 'question': 20}
n_participants = 40
n_items = 5
n_questions = 4
task = make_tasks(100, 350, n_questions)
task_effects = {
    'word':  task,
    'image': task,
}

# task_effects = {
#     'word':  make_tasks(100, 350, n_questions), #task,
#     'image': make_tasks(200, 500, n_questions)#task,
# }

df = simulate(
    design=design,
    word=word_params,
    image=image_params,
    noise=noise_params,
    n_participants=n_participants,
    n_items=n_items,
    n_questions=n_questions,
    task=task_effects
)
print(df.head())

   participant       group item_type  question_idx  item_idx          RT
0            0  both__allQ      word             0         0  351.028601
1            0  both__allQ     image             0         0  299.505542
2            0  both__allQ      word             0         1  274.889947
3            0  both__allQ     image             0         1  230.073288
4            0  both__allQ      word             0         2  287.701950


In [6]:
print_deltas(df)

Q0: | word - image | = 30.93
Q1: | word - image | = 28.91
Q2: | word - image | = 28.79
Q3: | word - image | = 29.06
